# ***MNIST Example***

In [1]:
%pip install numpy matplotlib idx2numpy

In [2]:
import numpy as np
import idx2numpy
import matplotlib.pyplot as plt
import numpy.linalg as inv

In [3]:
X_train = idx2numpy.convert_from_file('./dataset/train-images-idx3-ubyte')
X_test = idx2numpy.convert_from_file('./dataset/t10k-images-idx3-ubyte')
y_train = idx2numpy.convert_from_file('./dataset/train-labels-idx1-ubyte')
y_test = idx2numpy.convert_from_file('./dataset/t10k-labels-idx1-ubyte')

In [4]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(60000, 28, 28)
(10000, 28, 28)
(60000,)
(10000,)
